In [1]:
import pickle
import random
import math
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
#from sklearn.mixture import GMM
from sklearn.mixture import GaussianMixture
from sklearn.datasets import load_boston
from datetime import datetime
from datetime import timedelta
%matplotlib inline
# plt.matplotlib.rcParams.update({'font.size': 50})
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (12.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'   
plt.rcParams["patch.force_edgecolor"] = False
plt.rc('figure', titlesize=25)

In [2]:
PICKLE_PATH = '../augmented_datasets/pickles/hopkins_conf_gf0904_GDP_urban_weather.pkl'

In [3]:
hopkins_confirmed = pd.read_pickle(PICKLE_PATH)

In [4]:
hopkins_confirmed.describe()

,GDP,Urbanization,avg_m_tmp,avg_m_RH,avg_m_precip,avg_m_wind,Max_Cases,first_7,avg_interval_tmp,avg_interval_RH,...,3/29/2020,3/30/2020,3/31/2020,4/1/2020,4/2/2020,4/3/2020,4/4/2020,4/5/2020,4/6/2020,4/7/2020
count,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1070.000000,1063.000000,1070.000000,1070.000000,...,5295.000000,5308.000000,5322.000000,5336.000000,5336.000000,5328.000000,5320.000000,5331.000000,5327.000000,5331.000000
mean,52102.320042,74.298066,9.428750,70.922285,1.788514,11.821473,1686.908411,2.956743,12.753437,70.019905,...,180.928716,194.286890,214.074301,230.367822,250.538253,272.385893,298.439432,317.408923,336.277528,357.665864
std,21063.174970,14.606890,7.587477,9.037325,1.682517,3.680288,14944.506938,1.775053,7.341610,10.104754,...,3067.974341,3365.232865,3733.095866,4097.744440,4518.005728,4970.220197,5494.354993,5883.691669,6295.815448,6725.900356
min,396.000000,14.338000,-16.824675,11.831169,0.000000,3.007246,20.000000,0.000000,-9.926087,8.444444,...,-13.200000,-12.600000,-12.900000,-14.700000,-18.700000,-18.100000,-13.800000,-10.200000,-8.700000,-10.300000
25%,46609.000000,66.300000,3.945685,66.922078,0.032922,9.324675,36.250000,1.707143,6.865476,66.542857,...,6.350000,5.500000,6.100000,5.300000,4.300000,4.475000,4.500000,5.000000,5.200000,7.000000
50%,55172.000000,75.100000,8.482468,72.571429,1.601016,11.419895,86.500000,2.666667,12.008036,71.666667,...,16.000000,14.100000,14.000000,13.900000,15.000000,15.800000,14.995000,15.100000,16.800000,18.800000
75%,61594.000000,86.200000,13.866234,76.493506,2.681818,14.035146,306.750000,4.000000,17.847756,76.250940,...,51.000000,47.000000,59.000000,58.000000,54.000000,54.000000,60.000000,64.000000,60.000000,64.000000
max,200277.000000,100.000000,32.323377,88.608696,7.818052,29.774026,396223.000000,11.000000,35.100000,94.000000,...,140909.000000,161837.000000,188172.000000,213372.000000,243616.000000,275586.000000,308850.000000,337072.000000,366667.000000,396223.000000


In [5]:
def preprocess(X, y):
    """
    Perform mean normalization on the features and true labels.
    """
    X = (X - X.mean()) / (X.max() - X.min())
    y = (y - y.mean()) / (y.max() - y.min())
    return X, y

def compute_cost(X, y, theta):
    """
    Computes the average squared difference between an obserbation's actual and
    predicted values for linear regression.  
    """   
    J = 0
    m = X.shape[0]
    J = (1 / ( 2 * m )) * ((( (theta * X).sum(axis=1) - y) ** 2 ).sum() )
    return J

def gradient_descent(X, y, theta, alpha, num_iters):
    """
    Learn the parameters of the model using gradient descent using 
    the *training set*. Gradient descent is an optimization algorithm 
    used to minimize some (loss) function by iteratively moving in 
    the direction of steepest descent as defined by the negative of 
    the gradient. We use gradient descent to update the parameters
    (weights) of our model.
    """
    J_history = []
    theta = theta.copy()
    m = X.shape[0]
    for iter_ in range(num_iters):
        tmp_theta = theta.copy()
        for j in range(theta.shape[0]):
            tmp_theta[j] = theta[j] - (alpha /  m ) * ( ((theta * X).sum(axis=1) - y) * X.T[j]).sum() 
        theta = tmp_theta.copy()
        J_history.append(compute_cost(X, y, theta))
    return tmp_theta, J_history

def pinv(X, y):
    """
    Calculate the optimal values of the parameters using the pseudoinverse
    approach as you saw in class using the *training set*.
    """
    pinv_theta = []
    pinv_theta = (np.linalg.inv(X_train.T @ X_train) @ X_train.T) @ y_train
    return pinv_theta

def efficient_gradient_descent(X, y, theta, alpha, num_iters):   
    J_history = [] # Use a python list to save cost in every iteration
    theta = theta.copy() # avoid changing the original thetas
    THRESHOLD = 10 ** -8
    m = X.shape[0]
    i = 0
    while i <  num_iters:
        tmp_theta = theta.copy()
        for j in range(theta.shape[0]):
            tmp_theta[j] = theta[j] - (alpha /  m ) * ( ((theta * X).sum(axis=1) - y) * X.T[j]).sum() 
        theta = tmp_theta.copy()
        cost = compute_cost(X, y, theta)
        if i > 0:
            if (J_history[-1] - cost) < THRESHOLD:
                i = num_iters + 1
        J_history.append(cost)
        i += 1
    return theta, J_history

def find_best_alpha(X_train, y_train, X_val, y_val, iterations):
    """
    Iterate over provided values of alpha and train a model using the 
    *training* dataset. maintain a python dictionary with alpha as the 
    key and the loss on the *validation* set as the value.
    """
    alphas = [0.00001, 0.00003, 0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 2, 3]
    alpha_dict = {}
    np.random.seed(42) 
    theta = np.random.random(size=2)
    for alpha in alphas:
        computed_theta, _ = efficient_gradient_descent(X_train, y_train, theta, alpha, iterations)
        alpha_dict[alpha] = compute_cost(X_val, y_val, computed_theta)
    return alpha_dict

###### Preprocessing

Cut off of outlier data

In [66]:
pass

In [ ]:
feature_columns = ['GDP', 'Urbanization', 'avg_interval_tmp', 'avg_interval_RH']
target_columns = ['first_7', 'GF_Q1', 'GF_Q2', 'GF_Q3']
X = hopkins_confirmed[feature_columns].copy().dropna().values
y = hopkins_confirmed[target_columns].copy().dropna().values
len(hopkins_confirmed[feature_columns])
len(hopkins_confirmed[target_columns])

# Now we see that X != y
len(X)
len(y)


temp_hopkins = hopkins_confirmed.dropna().copy()
X = temp_hopkins[feature_columns].copy().dropna().values
y = temp_hopkins[target_columns].copy().dropna().values
len(X)
len(y)

In [17]:
feature_columns = ['GDP', 'Urbanization', 'avg_interval_tmp', 'avg_interval_RH']
target_columns = ['first_7', 'GF_Q1', 'GF_Q2', 'GF_Q3']
X = hopkins_confirmed[feature_columns].copy().dropna().values
y = hopkins_confirmed[target_columns].copy().dropna().values

It appears we have some nan values in some of the target columns, so we provide a temporary walkaround

In [72]:
len(hopkins_confirmed[feature_columns])
len(hopkins_confirmed[target_columns])
len(X)
len(y)

temp_hopkins = hopkins_confirmed.dropna().copy()
X = temp_hopkins[feature_columns].copy().dropna().values
y = temp_hopkins[target_columns].copy().dropna().values
len(X)
len(y)

5350

5350

926

926

926

926

###### Regression - Interval temp vs first_7 only

###### Regression - All params vs first_7

In [ ]:
from sklearn.model_selection import KFold
from sklearn import tree, metrics

def train_test_split(X, n_splits=5):
    '''
    Splits rows into training indices and test indices.
    :param X: numpy array of training data, e.g.  np.array([[1, 2], [3, 4], [1, 2], [3, 4]]) - each sample has two features
    :return: Returns indices of rows for train and test for n_splits. e.g. n_splits=2: 
    train_folds = [[0,2,3], [1,2,3]] test_folds = [[1], [0]] 
    '''

    kf = KFold(n_splits=n_splits, random_state=2346, shuffle=True)
    kf.get_n_splits(X)
    
    train_folds, test_folds = [], []
    
    for train_index, test_index in kf.split(X):
        train_folds.append(train_index)
        test_folds.append(test_index)
    
    return train_folds, test_folds


def decision_tree_train(X_train, y_train):
    dt = tree.DecisionTreeRegressor()
    trained_model = dt.fit(X_train, y_train)
    return trained_model


def linear_regression_train(X_train, y_train):
    # TODO - can use sklearn linear regression package. Should also have fit (like in decision_tree_train)
    return trained_model


## Example: ##

# Settings:

# Desired data we wish to train-test on - needs to be a pandas data frame formatted like 'colds' or 'hots':
data = colds
feature_cols = ['avg_interval_tmp']  # TODO - try adding more features (GDP, urban data...)
label_col = ['GF_Q3']

# Model function
model_fn = decision_tree_train  # can be replaced by linear regression

# Metric used to assess the model:
metric_fn = metrics.mean_squared_error

# Training-testing (nothing to change beyond this point):


X, y = np.array(data[feature_cols]), np.array(data[label_col])

# Split the data into train, test for n_splits train-test rounds
train_folds, test_folds = train_test_split(X, n_splits=5)

# Train-test the model for each of the n_splits:
for train_test_round in range(len(train_folds)):
    train_index = train_folds[train_test_round]
    test_index = test_folds[train_test_round]
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    dt_trained = decision_tree_train(X_train, y_train)
    if model_fn == decision_tree_train: 
        tree.plot_tree(dt_trained, feature_names=feature_cols)
    # evaluate on test
    y_pred = dt_trained.predict(X_test)
    print(metric_fn(y_test, y_pred))